# Natural Language Processing
One of my objectives was to predict from the description how well the movie was going to be received.
To do that I need to work with the ``description`` column as input, and change the ``rating`` column into a boolean one where True is over 5 of valoration.

In [19]:
import pandas as pd
import numpy as np

In [20]:
data = pd.read_csv('data\imdb_processed.csv')

In [21]:
sample = data.sample(100, random_state=42)

## Target column

In [22]:
def approved(rating):
    if rating >= 5.0:
        return True
    else:
        return False

In [23]:
data['rating_bool'] = data.rating.map(approved)
data['rating_bool'].value_counts()

True     34685
False     5101
Name: rating_bool, dtype: int64

The target is very unbalanced, and I also need to reduce the amount of rows in me dataset due to time constraints, so I will apply some downsampling.

In [24]:
from sklearn.utils import resample

In [25]:
category_0 = data[data['rating_bool'] == False]
print(category_0.shape)
category_1 = data[data['rating_bool'] == True]
print(category_1.shape)

(5101, 12)
(34685, 12)


In [26]:
category_1_undersampled = resample(category_1, 
                                   replace=False, 
                                   n_samples = len(category_0))

In [27]:
print(category_1_undersampled.shape)

(5101, 12)


In [28]:
data_downsampled = pd.concat([category_0, category_1_undersampled], axis=0)

In [29]:
data_downsampled.shape

(10202, 12)

## Processing text

In [30]:
# text = data[['genres', 'description']]
descriptions = data_downsampled.description


In [31]:
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

from nltk.corpus import stopwords

import re



In [32]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    reg_url = '(?:(?:https?|ftp):\/\/)?[\w\/\-?=%.]+\.[\w\/\-&?=%.]+'
    reg_sp  = '[^A-Za-z ]'

    s = re.sub(reg_url,'',s) 

    s = re.sub(reg_sp,' ',s) 

    return s

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    s = word_tokenize(s)

    return s

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = PorterStemmer() #I'm not convinced these go here
    lemmatizer = WordNetLemmatizer() 

    list = [lemmatizer.lemmatize(ps.stem(word)) for word in l]

    for word in l:
        word = ps.stem(word)
        word = lemmatizer.lemmatize(word)

    return list

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    clean_list = [word for word in l if not word in stopwords.words()]

    return clean_list

def full_process(s):
    '''
    Args:
        s: the string to process

    Returns:
        The list of words after removing the stopwords
    '''
    s = clean_up(s)
    l = tokenize(s)
    l = stem_and_lemmatize(l)
    clean_list = remove_stopwords(l)

    return clean_list

In [33]:
data_downsampled['tokens'] = descriptions.apply(full_process) #full
data_downsampled['tokens'][:15]

5      [guest, hous, stockholm, group, music, student...
9      [waitress, inga, pregnant, father, sailor, edv...
10     [john, ulla, happili, marri, live, villa, frid...
27     [tex, robbin, texa, ranger, pose, wolf, cassid...
40     [sensat, court, poverti, row, entri, town, vam...
47     [origin, titl, trail, killer, qv, misnom, wa, ...
122    [lisa, young, woman, live, countrysid, go, sto...
137    [gangster, get, boxer, throw, import, fight, f...
146                  [g, unemploy, join, gang, smuggler]
176    [timid, wynn, thrust, spotlight, hi, father, h...
182    [canadian, mounti, assign, bring, hi, brother,...
185    [kelli, daughter, fall, revenu, agent, hi, div...
195    [tad, wallac, act, flop, broadway, join, troop...
258    [pilot, hi, crash, land, island, run, psycho, ...
269    [sham, ceremoni, hall, dimwit, fridolf, marri,...
Name: tokens, dtype: object

In [34]:
data_downsampled

,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,rating,genres,mpaa,description,votes,year_process,rating_bool,tokens
5,tt0023534,En stulen vals,0,1932,83,3.9,['drama'],Not Rated,['A guest house in Stockholm. A group of music...,9.0,1932,False,"[guest, hous, stockholm, group, music, student..."
9,tt0023541,Svarta rosor,0,1932,86,4.8,['drama'],Not Rated,['The waitress Inga is pregnant and the father...,9.0,1932,False,"[waitress, inga, pregnant, father, sailor, edv..."
10,tt0023542,Svärmor kommer,0,1932,84,3.8,['comedy'],Not Rated,['John and Ulla are happily married and live i...,8.0,1932,False,"[john, ulla, happili, marri, live, villa, frid..."
27,tt0023570,The Texas Tornado,0,1932,53,4.7,['western'],Approved,"['Tex Robbins, a Texas Ranger, posing as ""Wolf...",25.0,1932,False,"[tex, robbin, texa, ranger, pose, wolf, cassid..."
40,tt0023592,Thrill of Youth,0,1932,60,3.9,['drama'],Not Rated,['A sensation-courting poverty-row entry in wh...,24.0,1932,False,"[sensat, court, poverti, row, entri, town, vam..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14646,tt0049476,Man in the Vault,0,1956,73,6.0,"['crime', 'drama', 'film-noir']",Approved,['A locksmith gets caught up in a bank robbery...,417.0,1956,True,"[locksmith, get, caught, bank, robberi, angel,..."
28517,tt0796926,Cheese Head,0,2006,90,6.0,"['comedy', 'drama']",Not Rated,['Year 1993. Ariel and his family spend their ...,502.0,2006,True,"[year, ariel, hi, famili, spend, summer, idyl,..."
14956,tt0049970,Yaqui Drums,0,1956,71,5.9,['western'],Not Rated,"[""After his brother's mysterious murder, Webb ...",22.0,1956,True,"[hi, brother, mysteri, murder, webb, dunham, h..."
9210,tt0040087,All My Sons,0,1948,94,7.4,"['drama', 'film-noir']",Approved,"[""During WW2, industrialist Joe Keller commits...",1978.0,1948,True,"[dure, ww, industrialist, joe, keller, commit,..."


In [36]:
data_downsampled.to_csv('data\data_downsampled.csv', index=False)

## Bag of Words
Is a distribution of frequencies for the words.

In [35]:
from nltk.probability import FreqDist